In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization


images = np.load('/home/s3963616/IDL/images_L.npy')
labels = np.load('/home/s3963616/IDL/labels.npy')
print(labels)
print(images.shape)

categories = np.zeros_like(labels[:, 0])

for i in range(12):
   
    categories[(labels[:, 0] == i) & (labels[:, 1] < 30)] = 2 * i
    categories[(labels[:, 0] == i) & (labels[:, 1] >= 30)] = 2 * i + 1

print(categories)

2023-11-10 06:55:09.923117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 06:55:10.121993: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-10 06:55:10.954578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64
2023-11-10 06:55:10.954671: W tensorflow/compiler/xla/stream

[[ 0  0]
 [ 0  0]
 [ 0  0]
 ...
 [11 59]
 [11 59]
 [11 59]]
(18000, 150, 150)
[ 0  0  0 ... 23 23 23]


In [2]:
# start to train
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from keras.layers import BatchNormalization

images_expanded = np.expand_dims(images, -1)
images_expanded = images_expanded.astype('float32') / 255.0

X_train, X_test, y_train, y_test = train_test_split(images_expanded, categories, test_size=0.2, random_state=42)



num_classes = len(np.unique(categories))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),  
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(32, (3, 3), activation='relu'),  
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(256, activation='relu'),  
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)




model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


model.fit(X_train, y_train, batch_size=128, epochs=30, validation_split=0.2)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)


print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


def calculate_time_difference(true_category, pred_category):
    
    true_minutes = (true_category // 6) * 60 + (true_category % 6) * 10
    pred_minutes = (pred_category // 6) * 60 + (pred_category % 6) * 10
    
    diff = abs(true_minutes - pred_minutes)
    
    if diff > 360:  
        diff = 720 - diff
    return diff


def common_sense_accuracy(y_true_classes, y_pred_classes):
    total_samples = len(y_true_classes)
    total_difference = 0
    for i in range(total_samples):
        total_difference += calculate_time_difference(y_true_classes[i], y_pred_classes[i])
    
    mean_difference = total_difference / total_samples
    return mean_difference


mean_diff = common_sense_accuracy(y_test_classes, y_pred_classes)
print(f"Common Sense Accuracy (Mean Time Difference): {mean_diff} minutes")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      160       
                                                                 
 batch_normalization (BatchN  (None, 148, 148, 16)     64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 batch_normalization_1 (Batc  (None, 72, 72, 32)       128       
 hNormalization)                                                 
                                                        

2023-11-10 05:08:19.796720: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 05:08:23.755159: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9375 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6
2023-11-10 05:08:23.755794: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 782 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


Epoch 1/30


2023-11-10 05:08:27.591230: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8800
2023-11-10 05:08:29.067280: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-10 05:08:29.074643: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f41e7676aa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-10 05:08:29.074671: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-10 05:08:29.074677: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-10 05:08:29.081045: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-10 

90/90 [==============================] - 10s 36ms/step - loss: 3.2812 - accuracy: 0.0467 - val_loss: 5.7247 - val_accuracy: 0.0413
Epoch 2/30
90/90 [==============================] - 3s 29ms/step - loss: 3.1556 - accuracy: 0.0498 - val_loss: 6.3799 - val_accuracy: 0.0413
Epoch 3/30
90/90 [==============================] - 2s 26ms/step - loss: 3.0517 - accuracy: 0.0698 - val_loss: 4.0791 - val_accuracy: 0.0413
Epoch 4/30
90/90 [==============================] - 3s 29ms/step - loss: 2.8758 - accuracy: 0.0946 - val_loss: 3.1061 - val_accuracy: 0.0750
Epoch 5/30
90/90 [==============================] - 3s 29ms/step - loss: 2.6859 - accuracy: 0.1374 - val_loss: 3.0587 - val_accuracy: 0.0917
Epoch 6/30
90/90 [==============================] - 3s 28ms/step - loss: 2.4733 - accuracy: 0.1703 - val_loss: 2.4976 - val_accuracy: 0.1872
Epoch 7/30
90/90 [==============================] - 3s 29ms/step - loss: 2.2881 - accuracy: 0.2134 - val_loss: 2.6008 - val_accuracy: 0.1837
Epoch 8/30
90/90 [=====

In [ ]:
# Now we transfer labels in 72 categories, every ten minutes

In [3]:
categories_10min = np.zeros_like(labels[:, 0])

for hour in range(12):  
    for ten_minute_mark in range(6): 
        start_min = ten_minute_mark * 10
        end_min = (ten_minute_mark + 1) * 10
        categories_10min[(labels[:, 0] == hour) & (labels[:, 1] >= start_min) & (labels[:, 1] < end_min)] = hour * 6 + ten_minute_mark

print(categories_10min, len(categories_10min))


[ 0  0  0 ... 71 71 71] 18000


In [ ]:
# dense 128 - 256 -512 -1024 

In [17]:
# start to train
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from keras.layers import BatchNormalization

images_expanded = np.expand_dims(images, -1)
images_expanded = images_expanded.astype('float32') / 255.0
# switch labels
X_train, X_test, y_train, y_test = train_test_split(images_expanded, categories_10min, test_size=0.2, random_state=42)


num_classes = len(np.unique(categories_10min))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),  
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(48, (3, 3), activation='relu'), 
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(1024, activation='relu'),  
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)


model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, batch_size=128, epochs=30, validation_split=0.2)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

def calculate_time_difference(true_category, pred_category):
    true_minutes = (true_category // 6) * 60 + (true_category % 6) * 10
    pred_minutes = (pred_category // 6) * 60 + (pred_category % 6) * 10
    diff = abs(true_minutes - pred_minutes)
    
    if diff > 360:  
        diff = 720 - diff
    return diff


def common_sense_accuracy(y_true_classes, y_pred_classes):
    total_samples = len(y_true_classes)
    total_difference = 0
    for i in range(total_samples):
        total_difference += calculate_time_difference(y_true_classes[i], y_pred_classes[i])
    mean_difference = total_difference / total_samples
    return mean_difference


test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)


print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

mean_diff = common_sense_accuracy(y_test_classes, y_pred_classes)
print(f"Common Sense Accuracy (Mean Time Difference): {mean_diff} minutes")


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_55 (Conv2D)          (None, 148, 148, 16)      160       
                                                                 
 batch_normalization_55 (Bat  (None, 148, 148, 16)     64        
 chNormalization)                                                
                                                                 
 max_pooling2d_55 (MaxPoolin  (None, 74, 74, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_56 (Conv2D)          (None, 72, 72, 32)        4640      
                                                                 
 batch_normalization_56 (Bat  (None, 72, 72, 32)       128       
 chNormalization)                                                
                                                     

In [ ]:
# Now we transfer labels into 720 categories, which are huge and the accuracy is relatively low without saying

In [3]:
categories_1min = np.zeros_like(labels[:, 0])

for hour in range(12):  
    for minute in range(60):  
        categories_1min[(labels[:, 0] == hour) & (labels[:, 1] == minute)] = hour * 60 + minute

print(categories_1min, len(categories_1min))

[  0   0   0 ... 719 719 719] 18000


In [12]:
# start to train
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from keras.layers import BatchNormalization

images_expanded = np.expand_dims(images, -1)
images_expanded = images_expanded.astype('float32') / 255.0


X_train, X_test, y_train, y_test = train_test_split(images_expanded, categories_1min, test_size=0.2, random_state=42)


num_classes = len(np.unique(categories_1min))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'), 
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'), 
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(256, (3, 3), activation='relu'),  
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'), 
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)



model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, batch_size=128, epochs=30, validation_split=0.2)


y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


def calculate_time_difference(true_category, pred_category):
    
    true_minutes = (true_category // 6) * 60 + (true_category % 6) * 10
    pred_minutes = (pred_category // 6) * 60 + (pred_category % 6) * 10
   
    diff = abs(true_minutes - pred_minutes)
   
    if diff > 360:  
        diff = 720 - diff
    return diff


def common_sense_accuracy(y_true_classes, y_pred_classes):
    total_samples = len(y_true_classes)
    total_difference = 0
    for i in range(total_samples):
        total_difference += calculate_time_difference(y_true_classes[i], y_pred_classes[i])

    mean_difference = total_difference / total_samples
    return mean_difference


test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)


print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


mean_diff = common_sense_accuracy(y_test_classes, y_pred_classes)
print(f"Common Sense Accuracy (Mean Time Difference): {mean_diff} minutes")



Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 148, 148, 16)      160       
                                                                 
 batch_normalization_35 (Bat  (None, 148, 148, 16)     64        
 chNormalization)                                                
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 74, 74, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 72, 72, 32)        4640      
                                                                 
 batch_normalization_36 (Bat  (None, 72, 72, 32)       128       
 chNormalization)                                                
                                                      

In [3]:
images = np.load('/home/s3963616/IDL/images_L.npy')
labels = np.load('/home/s3963616/IDL/labels.npy')
print(labels.shape)
print(images.shape)

(18000, 2)
(18000, 150, 150)


In [4]:
hours = labels[:, 0]
minutes = labels[:, 1] / 60 
continuous_labels = hours + minutes  
for i in range(10):
    print(continuous_labels[i])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [41]:
# start to train
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, Dense
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


images_expanded = np.expand_dims(images, -1)


X_train, X_test, y_train_continuous, y_test_continuous = train_test_split(images_expanded, continuous_labels, test_size=0.2, random_state=42)


X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


he_normal = HeNormal()
l1_l2_reg = l1_l2(l1=0.01, l2=0.001)
l2_reg = l2(0.001)  


model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu'),  
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(96, (3, 3), activation='relu'),  
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(1824, activation='relu'), 
    Dropout(0.3),
    
    Dense(1)  
])

adam_optimizer = Adam(learning_rate=0.001)



model.compile(optimizer=adam_optimizer, loss='mean_squared_error')

model.summary()


model.fit(X_train, y_train_continuous, batch_size=64, epochs=30, validation_split=0.2)


y_pred_continuous = model.predict(X_test).flatten()


def calculate_common_sense_error(y_true, y_pred):
    error = np.abs(y_true - y_pred)
    error = np.minimum(error, 24 - error)  
    error *= 60 
    return np.mean(error)


common_sense_error = calculate_common_sense_error(y_test_continuous, y_pred_continuous)
print(f"Common Sense Error: {common_sense_error:.2f} minutes")


test_loss = model.evaluate(X_test, y_test_continuous, verbose=0)
print(f"Test Loss: {test_loss:.4f}")

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_173 (Conv2D)         (None, 148, 148, 16)      160       
                                                                 
 batch_normalization_173 (Ba  (None, 148, 148, 16)     64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_173 (MaxPooli  (None, 74, 74, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_174 (Conv2D)         (None, 72, 72, 32)        4640      
                                                                 
 batch_normalization_174 (Ba  (None, 72, 72, 32)       128       
 tchNormalization)                                               
                                                     

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping


images = np.load('/home/s3963616/IDL/images_L.npy')
labels = np.load('/home/s3963616/IDL/labels.npy')


hours = labels[:, 0]
minutes = labels[:, 1] / 60 


images_expanded = np.expand_dims(images, -1)
images_expanded = images_expanded.astype('float32') / 255.0


X_train, X_test, y_train_hours, y_test_hours, y_train_minutes, y_test_minutes = train_test_split(
    images_expanded, hours, minutes, test_size=0.2, random_state=42
)


input_layer = Input(shape=(150, 150, 1))

x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(256, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)


x = Conv2D(128, (3, 3), activation='relu')(x)  
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)  



x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)


hours_output = Dense(1, activation='linear', name='hours_output')(x)
minutes_output = Dense(1, activation='linear', name='minutes_output')(x)


model = Model(inputs=input_layer, outputs=[hours_output, minutes_output])


model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'hours_output': 'mean_squared_error', 'minutes_output': 'mean_squared_error'},
              loss_weights={'hours_output': 0.5, 'minutes_output': 0.5})

model.summary()


model.fit(X_train, {'hours_output': y_train_hours, 'minutes_output': y_train_minutes},
          validation_split=0.2,
          batch_size=64,
          epochs=30)

predictions = model.predict(X_test)
predicted_hours, predicted_minutes = predictions[0], predictions[1]

predicted_hours_converted = predicted_hours * 12  
predicted_minutes_converted = predicted_minutes * 60  


total_error = 0
for i in range(len(predicted_hours_converted)):
    pred_hour = predicted_hours_converted[i][0]
    pred_minute = predicted_minutes_converted[i][0]
    actual_hour = y_test_hours[i] * 12
    actual_minute = y_test_minutes[i] * 60

    error = calculate_shortest_time_difference(pred_hour, pred_minute, actual_hour, actual_minute)
    total_error += error

average_error = total_error / len(predicted_hours_converted)
print(f"Average Time Difference Error: {average_error:.2f} minutes")

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 150, 150, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_75 (Conv2D)             (None, 148, 148, 32  320         ['input_16[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_75 (BatchN  (None, 148, 148, 32  128        ['conv2d_75[0][0]']              
 ormalization)                  )                                                          

Epoch 11/30
180/180 [==============================] - 5s 26ms/step - loss: 0.3974 - hours_output_loss: 0.7491 - minutes_output_loss: 0.0458 - val_loss: 3.0857 - val_hours_output_loss: 6.1076 - val_minutes_output_loss: 0.0637
Epoch 12/30
180/180 [==============================] - 5s 26ms/step - loss: 0.3647 - hours_output_loss: 0.6848 - minutes_output_loss: 0.0445 - val_loss: 2.7748 - val_hours_output_loss: 5.4833 - val_minutes_output_loss: 0.0662
Epoch 13/30
180/180 [==============================] - 5s 27ms/step - loss: 0.3687 - hours_output_loss: 0.6921 - minutes_output_loss: 0.0453 - val_loss: 1.8941 - val_hours_output_loss: 3.7335 - val_minutes_output_loss: 0.0547
Epoch 14/30
180/180 [==============================] - 5s 26ms/step - loss: 0.3514 - hours_output_loss: 0.6592 - minutes_output_loss: 0.0435 - val_loss: 0.8489 - val_hours_output_loss: 1.6424 - val_minutes_output_loss: 0.0555
Epoch 15/30
180/180 [==============================] - 5s 26ms/step - loss: 0.3223 - hours_outpu

In [5]:
def calculate_shortest_time_difference(predicted_hours, predicted_minutes, actual_hours, actual_minutes):

 
    predicted_hours %= 12
    actual_hours %= 12

    predicted_total_minutes = predicted_hours * 60 + predicted_minutes
    actual_total_minutes = actual_hours * 60 + actual_minutes


    minute_difference = abs(predicted_total_minutes - actual_total_minutes)

    
    shortest_difference = min(minute_difference, 720 - minute_difference)

    return shortest_difference


example1_shortest = calculate_shortest_time_difference(0, 10, 11, 50) 
example2_shortest = calculate_shortest_time_difference(0, 10, 9, 50)  

example1_shortest, example2_shortest



(20, 140)

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


images = np.load('/home/s3963616/IDL/images_L.npy')
labels = np.load('/home/s3963616/IDL/labels.npy')


hours = labels[:, 0]
minutes = labels[:, 1] / 60 


images_expanded = np.expand_dims(images, -1)
images_expanded = images_expanded.astype('float32') / 255.0


X_train, X_test, y_train_hours, y_test_hours, y_train_minutes, y_test_minutes = train_test_split(
    images_expanded, hours, minutes, test_size=0.2, random_state=42
)

y_train_hours_categorical = to_categorical(y_train_hours, num_classes=12)
y_test_hours_categorical = to_categorical(y_test_hours, num_classes=12)


input_layer = Input(shape=(150, 150, 1))

x = Conv2D(16, (3, 3), activation='relu')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)


x = Conv2D(90, (3, 3), activation='relu')(x) 
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)  



x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)


hours_output = Dense(12, activation='softmax', name='hours_output')(x)  
minutes_output = Dense(1, activation='linear', name='minutes_output')(x)


model = Model(inputs=input_layer, outputs=[hours_output, minutes_output])


model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'hours_output': 'categorical_crossentropy', 'minutes_output': 'mean_squared_error'},
              metrics={'hours_output': 'accuracy', 'minutes_output': 'mse'},
              loss_weights={'hours_output': 1.0, 'minutes_output': 1.0})

model.summary()


model.fit(X_train, {'hours_output': y_train_hours_categorical, 'minutes_output': y_train_minutes},
          validation_split=0.2,
          batch_size=64,
          epochs=30)

predictions = model.predict(X_test)
predicted_hours, predicted_minutes = predictions[0], predictions[1]


def calculate_shortest_time_difference(pred_hour, pred_minute, actual_hour, actual_minute):

 
    pred_hour %= 12
    actual_hour %= 12

    
    pred_total_minutes = pred_hour * 60 + pred_minute
    actual_total_minutes = actual_hour * 60 + actual_minute


    minute_difference = abs(pred_total_minutes - actual_total_minutes)

   
    shortest_difference = min(minute_difference, 720 - minute_difference)

    return shortest_difference


total_error = 0
for i in range(len(predicted_hours)):
    pred_hour = predicted_hours[i][0] * 12  
    pred_minute = predicted_minutes[i][0] * 60  
    actual_hour = y_test_hours[i] * 12
    actual_minute = y_test_minutes[i] * 60

    error = calculate_shortest_time_difference(pred_hour, pred_minute, actual_hour, actual_minute)
    total_error += error

average_error = total_error / len(predicted_hours)
print(f"Average Time Difference Error: {average_error:.2f} minutes")


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 150, 150, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_35 (Conv2D)             (None, 148, 148, 16  160         ['input_8[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_35 (BatchN  (None, 148, 148, 16  64         ['conv2d_35[0][0]']              
 ormalization)                  )                                                           

113/113 [==============================] - 1s 5ms/step
Average Time Difference Error: 8.98 minutes
